In [15]:
# import pandas as pd
# import re

# # === Load paths ===
# ideas_path = "/home/matias/Documents/media_monitor/data/idea_cluster/seed_ideas_exploded.jsonl"
# articles_path = "/home/matias/Documents/media_monitor/data/article_quotes/articles_exploded.jsonl"
# scraped_path = "/home/matias/Documents/media_monitor/data/scraped_links.jsonl"
# master_ref_path = "/home/matias/Documents/media_monitor/data/master_ref.csv"


# def enrich_idea_cluster():

#     # === Load data ===
#     df_ideas = pd.read_json(ideas_path, lines=True)
#     df_articles = pd.read_json(articles_path, lines=True)
#     df_scraped = pd.read_json(scraped_path, lines=True)
#     df_master = pd.read_csv(master_ref_path)

#     # === Normalize types ===
#     df_articles["article_id"] = df_articles["article_id"].astype(str)
#     df_scraped["article_id"] = df_scraped["article_id"].astype(str)
#     df_master["article_id"] = df_master["article_id"].astype(str)
#     df_ideas["source_ids"] = df_ideas["source_ids"].apply(lambda x: [str(i) for i in x])

#     # === Ensure consistent digest_file (strip seconds from id_digest) ===
#     # Drop seconds from id_digest to create digest_file (safe on nulls)
#     for df in [df_articles, df_ideas]:
#         df["digest_file"] = df["id_digest"].astype(str).str.extract(r"(.*T\d{4})")

#     # === Construct keys ===
#     df_articles["key"] = df_articles["digest_file"] + "::" + df_articles["article_id"]
#     # df_scraped["key"] = df_scraped["digest_file"] + "::" + df_scraped["article_id"]
#     df_master["key"] = df_master["digest_file"] + "::" + df_master["article_id"]

#     # === Explode ideas and build their keys ===
#     df_ideas_exploded = df_ideas.explode("source_ids").copy()
#     df_ideas_exploded["source_ids"] = df_ideas_exploded["source_ids"].astype(str)
#     df_ideas_exploded["key"] = df_ideas_exploded["digest_file"] + "::" + df_ideas_exploded["source_ids"]

#     # # === Enrich articles with index_id and metadata from master_ref ===
#     # merge_fields = ["index_id", "Source", "Title", "Published", "Link"]

#     # df_articles = df_articles.merge(
#     #     df_master[["index_id", "Source", "Title", "Published", "Link"]],
#     #     on="index_id", how="left"
#     # )

#     # === Enrich articles with scraped_data ===
#     df_articles["index_id"] = df_articles["index_id"].astype(str)
#     df_scraped["index_id"] = df_scraped["index_id"].astype(str)
#     df_articles = df_articles.drop("scraped_data", axis = 1).merge(
#         df_scraped[["index_id", "scraped_data"]],
#         on="index_id", how="left"
#     )

#     # === Enrich exploded ideas: attach article metadata via 'key' ===
#     df_ideas_exploded = df_ideas_exploded.merge(
#         df_articles[["key", "title", "cluster_topic", "index_id", "scraped_data"]],
#         on="key", how="left"
#     )

#     # Optionally return the enriched ideas
#     return df_ideas_exploded


# # Optional: save enriched ideas for downstream usage
# output_path = "/home/matias/Documents/media_monitor/data/idea_cluster/enriched_seed_ideas.jsonl"

# if __name__ == "__main__":
#     enriched_df = enrich_idea_cluster()
#     enriched_df.to_json(output_path, orient="records", lines=True, force_ascii=False)

In [16]:
# /home/matias/Documents/media_monitor/data/idea_cluster/test.jsonl
df = pd.read_json('./data/idea_cluster/test.jsonl', lines = True)


slice_df = df

idea_cols = slice_df.columns[slice_df.astype(str).nunique() == 1].tolist()
article_cols = slice_df.columns[slice_df.astype(str).nunique() > 1].tolist()


drop_noise = {"line_number", "source_file"}
idea_cols = [c for c in idea_cols if c not in drop_noise]
article_cols = [c for c in article_cols if c not in drop_noise]


In [17]:
def render_idea_block(row):
    out = []
    out.append(f"### 🧠 *{row.get('idea_title', 'Idea')}:*")
    if 'draft_editorial_angle' in row:
        out.append(f"**Enfoque propuesto:** {row['draft_editorial_angle']}")
    if 'key_facts' in row:
        out.append(f"**Hechos clave:** {row['key_facts']}")
    if 'potential_controversies' in row:
        out.append(f"**Controversias potenciales:** {row['potential_controversies']}")
    return "\n\n".join(out)

def render_article_block(row):
    parts = []
    if 'title' in row:
        parts.append(f"- 📰 *{row['title']}*")
    if 'scraped_data' in row and isinstance(row['scraped_data'], str):
        # parts.append(f"  {row['scraped_data'][:240].strip()}...")  # truncate if needed
        parts.append(f"  {row['scraped_data'].strip()}...")  # truncate if needed
    if 'index_id' in row:
        parts.append(f"  ([Ver fuente](https://news-link/{row['index_id']}))")  # adjust link if needed
    return "\n".join(parts)


In [18]:
def build_digest(slice_df):
    # Separate static + per-article info
    static_row = slice_df.iloc[0][idea_cols]

    # Prepare article rows
    article_data = (
        slice_df[article_cols]
        .drop_duplicates()
        .dropna(subset=["title"])  # Optional: avoid broken rows
    )

    idea_block = render_idea_block(static_row)
    article_blocks = "\n\n".join(render_article_block(r) for _, r in article_data.iterrows())

    return f"{idea_block}\n\n---\n\n{article_blocks}"


In [19]:
digest_md = build_digest(slice_df)
print(digest_md)


### 🧠 *Las emociones detrás de la condena: Lula y Cristina:*

**Enfoque propuesto:** Explorar cómo la relación personal entre Lula y Cristina se ve afectada por las circunstancias legales y políticas actuales, y qué implicaciones tiene esto para el futuro político de ambos.

**Hechos clave:** ['Lula reveló que Cristina lloró durante su conversación.', 'La condena de Cristina ha generado un impacto emocional en sus aliados.']

**Controversias potenciales:** ['¿Es la relación entre Lula y Cristina un factor determinante en la política argentina actual?', 'Debate sobre la manipulación emocional en la política.']

---

- 📰 *Lula da Silva dio detalles de su charla con Cristina Kirchner tras la condena a prisión*
  SECCIONES


INICIAR SESIÓN

Suscribite por $9.500 $900

LA NACION
>
El Mundo
Lula da Silva dio detalles de su charla con Cristina Kirchner tras la condena a prisión
El presidente de Brasil, que también estuvo detenido, expresó su solidaridad para con la exmandataria
21 de junio de